# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Noelle
from noelle import Motor, Fluid, FluidMixture

# Numpy
import numpy as np

# Matplotlib
from matplotlib import pyplot as plt
import matplotlib
import matplotlib as mpl
from labellines import labelLine, labelLines

# RocketCEA
from rocketcea.biprop_utils.rho_isp_plot_obj import RhoIspPlot

In [3]:
# Configure plot styles

# Sizes
mpl.rcParams['figure.figsize'] = [12.0, 6.0]
mpl.rcParams['figure.dpi'] = 120
mpl.rcParams['savefig.dpi'] = 120

# Font
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

# Style
plt.style.use(['science'])

# Fuel and Oxidizer Definitions

In [5]:
# Oxidizers
LOX = Fluid(name='O2(L)', coolprop_name='oxygen', formula='O 2', fluid_type='oxidizer', storage_pressure=35e5, storage_temperature=90.17)
GOX = Fluid(name='O2(G)', coolprop_name='oxygen', formula='O 2', fluid_type='oxidizer', storage_pressure=35e5, storage_temperature=298.15)
NOX =  Fluid(name='N2O', coolprop_name='NitrousOxide', formula=None, fluid_type='oxidizer', storage_temperature=298.15)

oxidizers_list = [GOX, LOX, NOX]

# Fuels
H2O = Fluid(name='H2O(L)', coolprop_name='water', formula='H 2 O 1', fluid_type='fuel', storage_pressure=35e5, storage_temperature=298.15)
LCH4 = Fluid(name='CH4(L)', coolprop_name='methane', formula='C 1 H 4', fluid_type='fuel', storage_pressure=1e5)
GCH4 = Fluid(name='CH4(G)', coolprop_name='methane', formula='C 1 H 4', fluid_type='fuel', storage_pressure=35e5, storage_temperature=298.15)
LC2H5OH = Fluid(name='C2H5OH(L)', coolprop_name='ethanol', formula='C 2 H 6 O 1', fluid_type='fuel', storage_pressure=35e5, storage_temperature=298.15)
JetA = Fluid(name='JetA', coolprop_name=None, formula=None, fluid_type='fuel', storage_pressure=35e5, storage_temperature=298.15, storage_density=815, storage_enthalpy=72466.6)

# Fuel Blend
H2O_30_C2H50H_70 = FluidMixture(fluid1=LC2H5OH, x1=70, fluid2=H2O, x2=30)

fuels_list = [LCH4, GCH4, LC2H5OH, JetA, H2O_30_C2H50H_70]

# Main Motor Parameters for Selected Propellants

In [6]:
NOELLE = Motor(
    NOX,
    H2O_30_C2H50H_70,
    thrust = 1500,
    burn_time = 10,
    p_chamber = 35,
    n_cstar = 1,
    n_cf = 1,
    cd_ox = 0.6,
    cd_fuel = 0.182,
    suboptimal = 1
)
NOELLE.report()

Thrust (N): 1500.00

Burn time (seconds): 10.00

Chamber pressure (bar): 35.0

Adiabatic chamber temperature (Kelvin): 3000.6

Molecular Weight of exhaust products (kg/kmol): 26.16

Ratio of specific heats of exhaust products: 1.14

Oxidiser/fuel mass ratio: 4.00

Combustion efficiency (%): 1.00

Thrust coefficient efficiency (%): 1.00

Pressure on oxidiser tank (bar): 56.52

Temperature on oxidiser tank (K): 298.15

Pressure on fuel tank (bar): 35.00

Temperature on fuel tank (K): 298.15

Characteristic velocity (m/s): 1532.24

Thrust coefficient: 1.52

Specific impulse (seconds): 238.06

Volumetric Specific impulse (Ns/m³): 1767686.65

Total mass flow rate (kg/s): 0.642

Oxidiser mass flow rate (kg/s): 0.514

Fuel mass flow rate (kg/s): 0.128

Total oxidiser mass (kg): 5.138

Total fuel mass (kg): 1.285

Nozzle throat diameter (mm): 18.9

Nozzle exit diameter (mm): 46.0



# Fuel and Oxidizers Comparison

Fuel List:
    - CH4 (L) - LNG
    - CH4 (G) - NG
    - C2H6O (L) - Ethanol
    - C12H23 (L) - JetA - Kerosene
    
Oxidizer List:
    - O2 (L)
    - O2 (G)
    - NO2 (L-G)

## Oxidizer Fuel Combinations

In [7]:
# Combinations
import itertools
oxidizer_fuel_combinations = list(itertools.product(oxidizers_list, fuels_list))

print("Number of combinations:", len(oxidizer_fuel_combinations))
for combination in oxidizer_fuel_combinations:
    print("-----------"+str(combination)+"-----------")
    test_motor = Motor(
        combination[0],
        combination[1],
        thrust = 1000,
        burn_time = 10,
        p_chamber = 35,
        n_cstar = 0.885,
        n_cf = 0.95,
        cd_ox = 0.4,
        cd_fuel = 0.4
    )
    test_motor.report()

Number of combinations: 15
-----------(O2(G), CH4(L))-----------
Ox name changed to GO2
Thrust (N): 1000.00

Burn time (seconds): 10.00

Chamber pressure (bar): 35.0

Adiabatic chamber temperature (Kelvin): 3460.0

Molecular Weight of exhaust products (kg/kmol): 20.46

Ratio of specific heats of exhaust products: 1.13

Oxidiser/fuel mass ratio: 3.10

Combustion efficiency (%): 0.89

Thrust coefficient efficiency (%): 0.95

Pressure on oxidiser tank (bar): 35.00

Temperature on oxidiser tank (K): 298.15

Pressure on fuel tank (bar): 1.00

Temperature on fuel tank (K): 111.51

Characteristic velocity (m/s): 1867.34

Thrust coefficient: 1.53

Specific impulse (seconds): 244.78

Volumetric Specific impulse (Ns/m³): 141552.61

Total mass flow rate (kg/s): 0.416

Oxidiser mass flow rate (kg/s): 0.315

Fuel mass flow rate (kg/s): 0.102

Total oxidiser mass (kg): 3.149

Total fuel mass (kg): 1.016

Nozzle throat diameter (mm): 15.8

Nozzle exit diameter (mm): 38.8

-----------(O2(G), CH4(G))--

In [8]:
# Combinations
import itertools
oxidizer_fuel_combinations = list(itertools.product(oxidizers_list, fuels_list))

for combination in oxidizer_fuel_combinations:
    # Extract fuel and oxidizer
    oxidizer = combination[0]
    fuel = combination[1]
    # Create test motor
    test_motor = Motor(oxidizer,
                        fuel,
                        thrust = 1000,
                        burn_time = 10,
                        p_chamber = 35,
                        n_cstar = 0.885,
                        n_cf = 0.95,
                        cd_ox = 0.4,
                        cd_fuel = 0.4)
    print("Test Motor - Oxidizer: *"+str(oxidizer)+"* | Fuel: *"+str(fuel)+"* | Isp: *{:0.1f}* s | Iv: *{:0.1f}* 10^3 Ns/m³ | c*: {:0.1f} m/s".format(test_motor.Isp, test_motor.Iv/100, test_motor.cstar))
    

Ox name changed to GO2
Test Motor - Oxidizer: *O2(G)* | Fuel: *CH4(L)* | Isp: *244.8* s | Iv: *1415.5* 10^3 Ns/m³ | c*: 1867.3 m/s
Ox name changed to GO2
Fuel name changed to GCH4
Test Motor - Oxidizer: *O2(G)* | Fuel: *CH4(G)* | Isp: *247.3* s | Iv: *910.6* 10^3 Ns/m³ | c*: 1887.9 m/s
Ox name changed to GO2
Test Motor - Oxidizer: *O2(G)* | Fuel: *C2H5OH(L)* | Isp: *228.1* s | Iv: *1585.2* 10^3 Ns/m³ | c*: 1737.1 m/s
Ox name changed to GO2
Test Motor - Oxidizer: *O2(G)* | Fuel: *JetA* | Isp: *235.7* s | Iv: *1460.6* 10^3 Ns/m³ | c*: 1800.0 m/s
Ox name changed to GO2
Test Motor - Oxidizer: *O2(G)* | Fuel: *C2H5OH(L)_70_H2O(L)_30* | Isp: *216.8* s | Iv: *1617.2* 10^3 Ns/m³ | c*: 1647.8 m/s
Test Motor - Oxidizer: *O2(L)* | Fuel: *CH4(L)* | Isp: *241.5* s | Iv: *19313.2* 10^3 Ns/m³ | c*: 1841.2 m/s
Fuel name changed to GCH4
Test Motor - Oxidizer: *O2(L)* | Fuel: *CH4(G)* | Isp: *244.1* s | Iv: *2218.4* 10^3 Ns/m³ | c*: 1861.9 m/s
Test Motor - Oxidizer: *O2(L)* | Fuel: *C2H5OH(L)* | Isp: *2

## $I_{sp}$ and $\rho \cdot I_{sp}$ Comparison

In [9]:
# Combinations
import itertools
oxidizer_fuel_combinations = list(itertools.product(oxidizers_list, fuels_list))

results_x = []
results_y = []

for combination in oxidizer_fuel_combinations:
    sub_result_x = []
    sub_result_y = []
    # Extract fuel and oxidizer
    oxidizer = combination[0]
    fuel = combination[1]
    for suboptimal in np.linspace(0.7, 1.3, 20):
        # Create test motor
        test_motor = Motor(oxidizer,
                            fuel,
                            thrust = 1000,
                            burn_time = 10,
                            p_chamber = 35,
                            n_cstar = 0.885,
                            n_cf = 0.95,
                            cd_ox = 0.4,
                            cd_fuel = 0.4,
                            suboptimal = suboptimal)
        sub_result_x.append(test_motor.propellant_storage_density)
        sub_result_y.append(test_motor.Isp)
    
    results_x.append(sub_result_x) 
    results_y.append(sub_result_y)
   
    print("Test Motor - Oxidizer: *"+str(oxidizer)+"* | Fuel: *"+str(fuel)+"* | Isp: *{:0.1f}* s | Iv: *{:0.1f}* 10^3 Ns/m³".format(test_motor.Isp, test_motor.Iv/100))

Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Ox name changed to GO2
Test Motor - Oxidizer: *O2(G)* | Fuel: *CH4(L)* | Isp: *234.9* s | Iv: *1292.0* 10^3 Ns/m³
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name changed to GO2
Fuel name changed to GCH4
Ox name 

In [10]:
plt.figure()
x_mesh = np.linspace(1, 1200, 1200)
# for i in range(40, 260, 20):
#     plt.plot(x_mesh, 1000*i/x_mesh, color='grey', linestyle='--', label=str(i))
# labelLines(plt.gca().get_lines(),zorder=2.5)

a5, = plt.plot(results_x[4], results_y[4], label=str(oxidizer_fuel_combinations[4])[1:-1])
a1, = plt.plot(results_x[6], results_y[6], label=str(oxidizer_fuel_combinations[6])[1:-1])
a2, = plt.plot(results_x[7], results_y[7], label=str(oxidizer_fuel_combinations[7])[1:-1])
a3, = plt.plot(results_x[8], results_y[8], label=str(oxidizer_fuel_combinations[8])[1:-1])
a5, = plt.plot(results_x[10], results_y[10], label=str(oxidizer_fuel_combinations[10])[1:-1])
a6, = plt.plot(results_x[11], results_y[11], label=str(oxidizer_fuel_combinations[11])[1:-1])
a1, = plt.plot(results_x[0], results_y[0], '--', label=str(oxidizer_fuel_combinations[0])[1:-1])
a2, = plt.plot(results_x[1], results_y[1], '--', label=str(oxidizer_fuel_combinations[1])[1:-1])
a3, = plt.plot(results_x[2], results_y[2], '--', label=str(oxidizer_fuel_combinations[2])[1:-1])
a4, = plt.plot(results_x[3], results_y[3], '--', label=str(oxidizer_fuel_combinations[3])[1:-1])
a6, = plt.plot(results_x[5], results_y[5], '--', label=str(oxidizer_fuel_combinations[5])[1:-1])
a4, = plt.plot(results_x[9], results_y[9], '--', label=str(oxidizer_fuel_combinations[9])[1:-1])

    
# plt.xlim(650, 1100)
# plt.ylim(190, 250)
plt.xlabel('Propellant Storage Density  (kg/m³)')
plt.ylabel('Specific Impulse  (s)')
plt.title('Specific Impulse vs. Density')
# plt.legend()
plt.grid()
plt.show()

# Export results to files
# plt.savefig("output.png", bbox_inches="tight")
# np.savetxt('I_sp_x.csv', results_x, delimiter=',')
# np.savetxt('I_sp_y.csv', results_y, delimiter=',')


RuntimeError: Failed to process string with tex because latex could not be found

In [ ]:
rp = RhoIspPlot(dpi=120, bipropL=oxidizer_fuel_combinations, Pc=500, eps=2.4)
rp.add_rho_isp_contours(label_frac_pos=0.4)
rp.show()